# Hoja de Trabajo \# 4

---


por Josué Obregón <br>
DS6011 - Feature Engineering <br>
UVG Masters - Escuela de Negocios<br>


## Objetivos

El objetivo de esta hoja de trabajo  es presentar al estudiante diferentes técnicas de codificación de variables categóricas. Éstas técnicas incluyen codificadores clásicos, de contraste y codificadores supervisados o bayesianos.

También se busca que el estudiante practique la utilización de éstas técnicas con las librerías disponibles en el lenguaje Python.


## Importación de librerías y carga de los datos a varios pandas [DataFrames](https://pandas.pydata.org/pandas-docs/version/1.1.5/reference/frame.html)




In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [41]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [42]:
import gdown

urls = ['https://drive.google.com/uc?export=download&id=16AGQw1nM9NYILv2aSZaSNSn9jBPByWPq', # okc_train  https://drive.google.com/file/d/16AGQw1nM9NYILv2aSZaSNSn9jBPByWPq/view?usp=sharing
        ]
outputs = ['okc_train.csv']
for url,output in zip(urls,outputs):
  gdown.download(url, f'data/{output}', quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=16AGQw1nM9NYILv2aSZaSNSn9jBPByWPq
To: /content/data/okc_train.csv
100%|██████████| 12.3M/12.3M [00:00<00:00, 23.3MB/s]


In [43]:
df = pd.read_csv('data/okc_train.csv',index_col=0)

In [44]:
df.head()

,age,diet,drinks,drugs,education,height,income,last_online,offspring,pets,...,hispanic_latin,indian,middle_eastern,native_american,other,pacific_islander,white,essay_length,Class,profile
1,22,strictly_anything,socially,never,working_on_college_university,75,missing,3,doesnt_have_kids_but_might_want_them,likes_dogs_and_likes_cats,...,0,0,0,0,0,0,1,3.420945,other,1
4,23,vegetarian,socially,drugs_missing,working_on_college_university,71,inc20000,3,doesnt_want_kids,likes_cats,...,0,0,0,0,0,0,1,2.671173,other,3
6,29,mostly_anything,socially,drugs_missing,graduated_from_college_university,67,missing,2,doesnt_have_kids_but_might_want_them,likes_cats,...,0,0,0,0,0,0,1,3.391112,stem,5
8,31,mostly_anything,socially,never,graduated_from_college_university,65,missing,2,doesnt_have_kids_but_wants_them,likes_dogs_and_likes_cats,...,0,0,0,0,0,0,1,3.090611,other,6
10,37,mostly_anything,not_at_all,never,working_on_two_year_college,65,missing,3,kids_missing,likes_dogs_and_likes_cats,...,0,0,0,0,0,0,1,3.382377,other,7


In [45]:
df['drinks'].value_counts()

socially          27521
rarely             3981
often              3413
not_at_all         2157
drinks_missing     1208
very_often          319
desperately         210
Name: drinks, dtype: int64

In [46]:
df['status'].value_counts()

single            36014
seeing_someone     1354
available          1234
married             204
unknown               3
Name: status, dtype: int64

Pequeño dataset de prueba para algunas explicaciones y para las figuras en las diapositivas


In [47]:
df_test = pd.DataFrame({
... 'City': ['SF', 'SF', 'SF', 'NYC', 'NYC', 'NYC',
... 'Seattle', 'Seattle', 'Seattle'],
... 'Rent': [3999, 4000, 4001, 3499, 3500, 3501, 2499, 2500, 2501]
... })

In [48]:
df_test

,City,Rent
0,SF,3999
1,SF,4000
2,SF,4001
3,NYC,3499
4,NYC,3500
5,NYC,3501
6,Seattle,2499
7,Seattle,2500
8,Seattle,2501


Para esta hoja de trabajo vamos a utilizar la librería [category_encoders](http://contrib.scikit-learn.org/category_encoders/index.html), la cual es compatible con scikit-learn.

In [49]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
col = 'drinks'

# Unsupervised Encoders

## Classic Encoders

### One-Hot Encoder

In [51]:
from category_encoders import OneHotEncoder

In [52]:
onehot_enc = OneHotEncoder(use_cat_names=True  )

In [53]:
onehot_enc.fit_transform(df[[col]])

,drinks_socially,drinks_not_at_all,drinks_rarely,drinks_often,drinks_drinks_missing,drinks_very_often,drinks_desperately
1,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0
6,1,0,0,0,0,0,0
8,1,0,0,0,0,0,0
10,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
59940,1,0,0,0,0,0,0
59943,0,0,0,1,0,0,0
59944,0,1,0,0,0,0,0
59945,1,0,0,0,0,0,0


In [54]:
onehot_enc.mapping[0]['mapping'] #   mapping[0]['mapping']

,drinks_socially,drinks_not_at_all,drinks_rarely,drinks_often,drinks_drinks_missing,drinks_very_often,drinks_desperately
1,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0
5,0,0,0,0,1,0,0
6,0,0,0,0,0,1,0
7,0,0,0,0,0,0,1
-1,0,0,0,0,0,0,0
-2,0,0,0,0,0,0,0


### Binary Encoder

In [55]:
from category_encoders import BinaryEncoder

In [56]:
bin_encoder = BinaryEncoder()

In [57]:
bin_encoder.fit_transform(df[col])

,drinks_0,drinks_1,drinks_2
1,0,0,1
4,0,0,1
6,0,0,1
8,0,0,1
10,0,1,0
...,...,...,...
59940,0,0,1
59943,1,0,0
59944,0,1,0
59945,0,0,1


In [58]:
bin_encoder.mapping

[{'col': 'drinks',
  'mapping':     drinks_0  drinks_1  drinks_2
   1         0         0         1
   2         0         1         0
   3         0         1         1
   4         1         0         0
   5         1         0         1
   6         1         1         0
   7         1         1         1
  -1         0         0         0
  -2         0         0         0}]

In [59]:
bin_encoder.mapping[0]

{'col': 'drinks',
 'mapping':     drinks_0  drinks_1  drinks_2
  1         0         0         1
  2         0         1         0
  3         0         1         1
  4         1         0         0
  5         1         0         1
  6         1         1         0
  7         1         1         1
 -1         0         0         0
 -2         0         0         0}

### Frequency or Count Encoder

In [60]:
from category_encoders import CountEncoder

In [61]:
count_enc = CountEncoder( )

In [62]:
count_enc.fit_transform(df[col])

,drinks
1,27521
4,27521
6,27521
8,27521
10,2157
...,...
59940,27521
59943,3413
59944,2157
59945,27521


In [63]:
count_enc.mapping

{'drinks': socially          27521
 rarely             3981
 often              3413
 not_at_all         2157
 drinks_missing     1208
 very_often          319
 desperately         210
 Name: drinks, dtype: int64}

In [64]:
df[col].value_counts()

socially          27521
rarely             3981
often              3413
not_at_all         2157
drinks_missing     1208
very_often          319
desperately         210
Name: drinks, dtype: int64

Intentemos utilizando las funciones de combinacion de grupos

In [65]:
count_enc = CountEncoder(combine_min_nan_groups=True, min_group_size=500, min_group_name='otros', normalize=True)

In [66]:
count_enc.fit_transform(df[col])

,drinks
1,0.709140
4,0.709140
6,0.709140
8,0.709140
10,0.055580
...,...
59940,0.709140
59943,0.087944
59944,0.055580
59945,0.709140


In [67]:
count_enc.mapping

{'drinks': socially          0.709140
 rarely            0.102579
 often             0.087944
 not_at_all        0.055580
 drinks_missing    0.031127
 otros             0.013631
 Name: drinks, dtype: float64}

### Ordinal Encoder

In [68]:
from category_encoders import OrdinalEncoder

In [69]:
ord_enc = OrdinalEncoder( )

In [70]:
ord_enc.fit_transform(df[col])

,drinks
1,1
4,1
6,1
8,1
10,2
...,...
59940,1
59943,4
59944,2
59945,1


In [71]:
ord_enc.category_mapping[0]['mapping']

socially          1
not_at_all        2
rarely            3
often             4
drinks_missing    5
very_often        6
desperately       7
NaN              -2
dtype: int64

In [72]:
drink_dict =  {None: 0, 'not_at_all': 1,
               'rarely': 2, 'socially': 3,
               'often': 4,'very_often': 5 ,
               'desperately': 6, 'drinks_missing':-1}
col_drink_dict = {'col': 'drinks', 'mapping': drink_dict}

In [73]:
ord_enc = OrdinalEncoder(mapping=[col_drink_dict] )

In [74]:
ord_enc.fit_transform(df[col], )

,drinks
1,3
4,3
6,3
8,3
10,1
...,...
59940,3
59943,4
59944,1
59945,3


In [75]:
ord_enc.category_mapping[0]['mapping']

None              0
not_at_all        1
rarely            2
socially          3
often             4
very_often        5
desperately       6
drinks_missing   -1
dtype: int64

###Feature Hashing

In [76]:
from sklearn.feature_extraction import FeatureHasher

In [77]:
col = 'where_town'

In [78]:
df[col].describe()

count             38809
unique               51
top       san_francisco
freq              20028
Name: where_town, dtype: object

In [79]:
df[col].head()

1     south_san_francisco
4                berkeley
6           san_francisco
8           san_francisco
10              san_mateo
Name: where_town, dtype: object

In [80]:
hash_enc = FeatureHasher(n_features=8, input_type='string', alternate_sign=True)

In [81]:
hashed_features = hash_enc.fit_transform([[x] for x in df[col]])

In [82]:
hashed_features.toarray()

array([[ 0.,  0.,  0., ..., -1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., -1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [83]:
np_hashed = np.array(hashed_features.todense())

In [84]:
np.unique(np_hashed,axis=0).shape

(16, 8)

In [85]:
np.unique(np_hashed,axis=0)

array([[-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [86]:
from sys import getsizeof

In [87]:
print('Our pandas Series, in bytes: ', getsizeof(df[col]))
print('Our hashed numpy array, in bytes: ', getsizeof(hashed_features))

Our pandas Series, in bytes:  2944478
Our hashed numpy array, in bytes:  48


## Contrast Encoders

### Diferencia entre One-hot Encoding y Dummy Coding

In [88]:
df_test

,City,Rent
0,SF,3999
1,SF,4000
2,SF,4001
3,NYC,3499
4,NYC,3500
5,NYC,3501
6,Seattle,2499
7,Seattle,2500
8,Seattle,2501


In [89]:
print('Global mean: ',df_test['Rent'].mean())
print(df_test.groupby('City').mean())

Global mean:  3333.3333333333335
           Rent
City           
NYC      3500.0
SF       4000.0
Seattle  2500.0


In [90]:
one_hot_df = pd.get_dummies( df_test, prefix=['city'] )

In [91]:
one_hot_df

,Rent,city_NYC,city_SF,city_Seattle
0,3999,0,1,0
1,4000,0,1,0
2,4001,0,1,0
3,3499,1,0,0
4,3500,1,0,0
5,3501,1,0,0
6,2499,0,0,1
7,2500,0,0,1
8,2501,0,0,1


In [92]:
one_hot_df[['city_NYC','city_SF','city_Seattle']].drop_duplicates()

,city_NYC,city_SF,city_Seattle
0,0,1,0
3,1,0,0
6,0,0,1


Con la codificación one-hot, el término de intersección (intercepto) representa la media global de la variable objetivo: 'Renta', y cada uno de los coeficientes lineales representa que tanto difiere la renta media de esa ciudad respecto a la media global.

In [93]:
from sklearn import linear_model

In [94]:
model = linear_model.LinearRegression()
model.fit(one_hot_df[['city_NYC', 'city_SF', 'city_Seattle']],one_hot_df['Rent'])

LinearRegression()

In [95]:
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Coefficients: [ 166.66666667  666.66666667 -833.33333333]
Intercept: 3333.3333333333335


In [96]:
print('Global mean: ',df_test['Rent'].mean())
print(df_test.groupby('City').mean())

Global mean:  3333.3333333333335
           Rent
City           
NYC      3500.0
SF       4000.0
Seattle  2500.0


Ahora entrenemos el mismo modelo con dummy coding

In [97]:
dummy_df = pd.get_dummies(df_test, prefix=['city'], drop_first=True)
dummy_df

,Rent,city_SF,city_Seattle
0,3999,1,0
1,4000,1,0
2,4001,1,0
3,3499,0,0
4,3500,0,0
5,3501,0,0
6,2499,0,1
7,2500,0,1
8,2501,0,1


Con codificación dummy, el coeficiente de sesgo (intercepto) representa el valor medio de la variable $y$ para la categoría de referencia, que en el ejemplo es la ciudad NYC. El coeficiente para la *i*-ésima característica es igual a la diferencia entre el valor medio de la respuesta para la i-ésima categoría y la media de la categoría de referencia.

In [98]:
model.fit(dummy_df[['city_SF', 'city_Seattle']], dummy_df['Rent'])
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Coefficients: [  500. -1000.]
Intercept: 3500.0


In [99]:
print('Global mean: ',df_test['Rent'].mean())
print(df_test.groupby('City').mean())

Global mean:  3333.3333333333335
           Rent
City           
NYC      3500.0
SF       4000.0
Seattle  2500.0


### Dummy Coding

In [100]:
col='drinks'

In [101]:
from sklearn.preprocessing import OneHotEncoder as OneHotEncoder_sk

In [102]:
dummy_enc =OneHotEncoder_sk(drop='first', sparse=False )

In [103]:
dummy_enc.fit_transform(df[[col]])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [104]:
dummy_enc.categories_

[array(['desperately', 'drinks_missing', 'not_at_all', 'often', 'rarely',
        'socially', 'very_often'], dtype=object)]

In [105]:
dummy_df2 = pd.get_dummies(df[col], prefix='dr', drop_first=True) #dummy_nabool
dummy_df2.head()

,dr_drinks_missing,dr_not_at_all,dr_often,dr_rarely,dr_socially,dr_very_often
1,0,0,0,0,1,0
4,0,0,0,0,1,0
6,0,0,0,0,1,0
8,0,0,0,0,1,0
10,0,1,0,0,0,0


In [106]:
dummy_df2.drop_duplicates()

,dr_drinks_missing,dr_not_at_all,dr_often,dr_rarely,dr_socially,dr_very_often
1,0,0,0,0,1,0
10,0,1,0,0,0,0
19,0,0,0,1,0,0
41,0,0,1,0,0,0
93,1,0,0,0,0,0
97,0,0,0,0,0,1
549,0,0,0,0,0,0



### Sum (or Deviation) Coding

In [107]:
from category_encoders import SumEncoder

In [108]:
sum_enc = SumEncoder()

In [109]:
sum_enc.fit_transform(df[[col]])

/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",


,intercept,drinks_0,drinks_1,drinks_2,drinks_3,drinks_4,drinks_5
1,1,1.0,0.0,0.0,0.0,0.0,0.0
4,1,1.0,0.0,0.0,0.0,0.0,0.0
6,1,1.0,0.0,0.0,0.0,0.0,0.0
8,1,1.0,0.0,0.0,0.0,0.0,0.0
10,1,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
59940,1,1.0,0.0,0.0,0.0,0.0,0.0
59943,1,0.0,0.0,0.0,1.0,0.0,0.0
59944,1,0.0,1.0,0.0,0.0,0.0,0.0
59945,1,1.0,0.0,0.0,0.0,0.0,0.0


In [110]:
sum_enc.mapping[0]['mapping']

,drinks_0,drinks_1,drinks_2,drinks_3,drinks_4,drinks_5
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,0.0,0.0,0.0,0.0,1.0
7,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
-1,0.0,0.0,0.0,0.0,0.0,0.0
-2,0.0,0.0,0.0,0.0,0.0,0.0


Utilizando el ejemplo de prueba

In [111]:
sum_enc = SumEncoder()
sum_df = sum_enc.fit_transform(df_test['City'])
sum_df

/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",


,intercept,City_0,City_1
0,1,1.0,0.0
1,1,1.0,0.0
2,1,1.0,0.0
3,1,0.0,1.0
4,1,0.0,1.0
5,1,0.0,1.0
6,1,-1.0,-1.0
7,1,-1.0,-1.0
8,1,-1.0,-1.0


In [112]:
sum_enc.mapping[0]['mapping']

,City_0,City_1
1,1.0,0.0
2,0.0,1.0
3,-1.0,-1.0
-1,0.0,0.0
-2,0.0,0.0


In [113]:
sum_enc.ordinal_encoder.category_mapping[0]['mapping']

SF         1
NYC        2
Seattle    3
NaN       -2
dtype: int64

In [114]:
sum_df['Rent']=df_test['Rent']
sum_df

,intercept,City_0,City_1,Rent
0,1,1.0,0.0,3999
1,1,1.0,0.0,4000
2,1,1.0,0.0,4001
3,1,0.0,1.0,3499
4,1,0.0,1.0,3500
5,1,0.0,1.0,3501
6,1,-1.0,-1.0,2499
7,1,-1.0,-1.0,2500
8,1,-1.0,-1.0,2501


In [115]:
model = linear_model.LinearRegression()
model.fit(sum_df[['City_0', 'City_1']],sum_df['Rent']) # 0 = SF, 1 = NYC

LinearRegression()

La codificación de efectos (sum coding) es muy similar a la codificación dummy, pero da como resultado modelos de regresión lineal que son aún más simples de interpretar.

En el ejemplo, vemos que el término de intersección representa la media global del la variable respuesta, y los coeficientes individuales indican cuánto las medias de las categorías individuales difieren de la media global. (Esto se llama el efecto principal de la categoría o nivel, de ahí el nombre "codificación de efectos").

La codificación One-hot, encontró la misma intersección y coeficientes, pero en ese caso hay coeficientes lineales para cada ciudad. En la codificación de efecto, ningun feature representa la categoría de referencia, ***por lo que el efecto de la categoría de referencia debe calcularse por separado como la suma negativa de los coeficientes de todas las demás categorías.***

In [116]:
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')
print(f'Negative sum of coaefficients:{np.sum(model.coef_*-1)}')

Coefficients: [666.66666667 166.66666667]
Intercept: 3333.3333333333335
Negative sum of coaefficients:-833.3333333333336


In [117]:
print('Global mean: ',df_test['Rent'].mean())
print(df_test.groupby('City').mean())

Global mean:  3333.3333333333335
           Rent
City           
NYC      3500.0
SF       4000.0
Seattle  2500.0


Restulados de la regresión lineal con One-hot encoding

Coefficients: $[ 166.66666667,   666.66666667,  -833.33333333]$

Intercept: $3333.3333333333335$

### Backward Difference Coding

In [118]:
from category_encoders import BackwardDifferenceEncoder

In [119]:
back_diff_enc = BackwardDifferenceEncoder()

In [120]:
back_diff_enc.fit_transform(df[[col]])

/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",


,intercept,drinks_0,drinks_1,drinks_2,drinks_3,drinks_4,drinks_5
1,1,-0.857143,-0.714286,-0.571429,-0.428571,-0.285714,-0.142857
4,1,-0.857143,-0.714286,-0.571429,-0.428571,-0.285714,-0.142857
6,1,-0.857143,-0.714286,-0.571429,-0.428571,-0.285714,-0.142857
8,1,-0.857143,-0.714286,-0.571429,-0.428571,-0.285714,-0.142857
10,1,0.142857,-0.714286,-0.571429,-0.428571,-0.285714,-0.142857
...,...,...,...,...,...,...,...
59940,1,-0.857143,-0.714286,-0.571429,-0.428571,-0.285714,-0.142857
59943,1,0.142857,0.285714,0.428571,-0.428571,-0.285714,-0.142857
59944,1,0.142857,-0.714286,-0.571429,-0.428571,-0.285714,-0.142857
59945,1,-0.857143,-0.714286,-0.571429,-0.428571,-0.285714,-0.142857


In [121]:
back_diff_enc.mapping[0]['mapping']

,drinks_0,drinks_1,drinks_2,drinks_3,drinks_4,drinks_5
1,-0.857143,-0.714286,-0.571429,-0.428571,-0.285714,-0.142857
2,0.142857,-0.714286,-0.571429,-0.428571,-0.285714,-0.142857
3,0.142857,0.285714,-0.571429,-0.428571,-0.285714,-0.142857
4,0.142857,0.285714,0.428571,-0.428571,-0.285714,-0.142857
5,0.142857,0.285714,0.428571,0.571429,-0.285714,-0.142857
6,0.142857,0.285714,0.428571,0.571429,0.714286,-0.142857
7,0.142857,0.285714,0.428571,0.571429,0.714286,0.857143
-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
-2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Utilizando el ejemplo de prueba

In [122]:
back_diff_enc = BackwardDifferenceEncoder(  )
back_diff_df = back_diff_enc.fit_transform(df_test['City'])
back_diff_df['Rent']=df_test['Rent']
back_diff_df

/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",


,intercept,City_0,City_1,Rent
0,1,-0.666667,-0.333333,3999
1,1,-0.666667,-0.333333,4000
2,1,-0.666667,-0.333333,4001
3,1,0.333333,-0.333333,3499
4,1,0.333333,-0.333333,3500
5,1,0.333333,-0.333333,3501
6,1,0.333333,0.666667,2499
7,1,0.333333,0.666667,2500
8,1,0.333333,0.666667,2501


In [123]:
back_diff_enc.mapping[0]['mapping']

,City_0,City_1
1,-0.666667,-0.333333
2,0.333333,-0.333333
3,0.333333,0.666667
-1,0.000000,0.000000
-2,0.000000,0.000000


In [124]:
back_diff_enc.ordinal_encoder.category_mapping[0]['mapping']

SF         1
NYC        2
Seattle    3
NaN       -2
dtype: int64

In [125]:
from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(back_diff_df[['City_0', 'City_1']],back_diff_df['Rent'])  # 0 = SF, 1 = NYC

LinearRegression()

La codificación de diferencia en reversa es útil para codificar variables ordinales.

En el ejemplo, vemos que el término de intersección representa la media global de la variable respuesta, y los coeficientes individuales indican cuánto las medias de las categorías individuales difieren de la media de la categoría inmediatamente anterior.

In [126]:
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Coefficients: [ -500. -1000.]
Intercept: 3333.3333333333335


In [127]:
print('Global mean: ',df_test['Rent'].mean())
print(df_test.groupby('City').mean())

Global mean:  3333.3333333333335
           Rent
City           
NYC      3500.0
SF       4000.0
Seattle  2500.0


### Helmert Coding

In [128]:
from category_encoders import HelmertEncoder

In [129]:
helm_enc = HelmertEncoder()

In [130]:
helm_enc.fit_transform(df[[col]])

/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",


,intercept,drinks_0,drinks_1,drinks_2,drinks_3,drinks_4,drinks_5
1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
10,1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...
59940,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
59943,1,0.0,0.0,3.0,-1.0,-1.0,-1.0
59944,1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
59945,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [131]:
helm_enc.mapping[0]['mapping']

,drinks_0,drinks_1,drinks_2,drinks_3,drinks_4,drinks_5
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,0.0,2.0,-1.0,-1.0,-1.0,-1.0
4,0.0,0.0,3.0,-1.0,-1.0,-1.0
5,0.0,0.0,0.0,4.0,-1.0,-1.0
6,0.0,0.0,0.0,0.0,5.0,-1.0
7,0.0,0.0,0.0,0.0,0.0,6.0
-1,0.0,0.0,0.0,0.0,0.0,0.0
-2,0.0,0.0,0.0,0.0,0.0,0.0


Utilizando el ejemplo de prueba

In [132]:
helm_enc = HelmertEncoder( )
helm_df = helm_enc.fit_transform(df_test['City'])
helm_df['Rent']=df_test['Rent']
helm_df

/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",


,intercept,City_0,City_1,Rent
0,1,-1.0,-1.0,3999
1,1,-1.0,-1.0,4000
2,1,-1.0,-1.0,4001
3,1,1.0,-1.0,3499
4,1,1.0,-1.0,3500
5,1,1.0,-1.0,3501
6,1,0.0,2.0,2499
7,1,0.0,2.0,2500
8,1,0.0,2.0,2501


In [133]:
helm_enc.mapping[0]['mapping']

,City_0,City_1
1,-1.0,-1.0
2,1.0,-1.0
3,0.0,2.0
-1,0.0,0.0
-2,0.0,0.0


In [134]:
model = linear_model.LinearRegression()
model.fit(helm_df[['City_0', 'City_1']],helm_df['Rent'])  # 0 = SF, 1 = NYC

LinearRegression()

In [135]:
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Coefficients: [-250.         -416.66666667]
Intercept: 3333.3333333333335


In [136]:
print('Global mean: ',df_test['Rent'].mean())
print(df_test.groupby('City').mean())

Global mean:  3333.3333333333335
           Rent
City           
NYC      3500.0
SF       4000.0
Seattle  2500.0


In [137]:
df_test[df_test['City']!='NYC']['Rent'].mean()

3250.0

In [138]:
df_test[df_test['City']!='Seattle']['Rent'].mean()

3750.0

#Supervised Encoders

In [139]:
col_cat = 'drinks'
col_num = 'essay_length'

In [140]:
from sklearn.preprocessing import LabelEncoder

In [141]:
label_enc = LabelEncoder()
df['Class_enc'] = label_enc.fit_transform(df['Class'])

In [142]:
df.head()

,age,diet,drinks,drugs,education,height,income,last_online,offspring,pets,...,indian,middle_eastern,native_american,other,pacific_islander,white,essay_length,Class,profile,Class_enc
1,22,strictly_anything,socially,never,working_on_college_university,75,missing,3,doesnt_have_kids_but_might_want_them,likes_dogs_and_likes_cats,...,0,0,0,0,0,1,3.420945,other,1,0
4,23,vegetarian,socially,drugs_missing,working_on_college_university,71,inc20000,3,doesnt_want_kids,likes_cats,...,0,0,0,0,0,1,2.671173,other,3,0
6,29,mostly_anything,socially,drugs_missing,graduated_from_college_university,67,missing,2,doesnt_have_kids_but_might_want_them,likes_cats,...,0,0,0,0,0,1,3.391112,stem,5,1
8,31,mostly_anything,socially,never,graduated_from_college_university,65,missing,2,doesnt_have_kids_but_wants_them,likes_dogs_and_likes_cats,...,0,0,0,0,0,1,3.090611,other,6,0
10,37,mostly_anything,not_at_all,never,working_on_two_year_college,65,missing,3,kids_missing,likes_dogs_and_likes_cats,...,0,0,0,0,0,1,3.382377,other,7,0


## Target Encoder

In [143]:
from category_encoders import TargetEncoder

In [144]:
target_enc = TargetEncoder() #min_samples_leaf = k, smoothing = f
target_enc.fit_transform(df[col_cat],df['Class_enc'])

,drinks
1,0.189201
4,0.189201
6,0.189201
8,0.189201
10,0.147891
...,...
59940,0.189201
59943,0.195429
59944,0.147891
59945,0.189201


In [145]:
target_enc.mapping

{'drinks': drinks
  1    0.189201
  2    0.147891
  3    0.170058
  4    0.195429
  5    0.179636
  6    0.137931
  7    0.171429
 -1    0.184674
 -2    0.184674
 dtype: float64}

In [146]:
target_enc = TargetEncoder( ) # min_samples_leaf=1 (k), smoothing=1.0 (f)
target_enc.fit_transform(df[col_cat],df[col_num])

,drinks
1,3.136058
4,3.136058
6,3.136058
8,3.136058
10,3.145403
...,...
59940,3.136058
59943,3.138155
59944,3.145403
59945,3.136058


In [147]:
target_enc.mapping

{'drinks': drinks
  1    3.136058
  2    3.145403
  3    3.164222
  4    3.138155
  5    2.963267
  6    2.915820
  7    3.072538
 -1    3.132119
 -2    3.132119
 dtype: float64}

## Leave-one-out Encoder

In [148]:
from category_encoders import LeaveOneOutEncoder

In [149]:
lou_enc = LeaveOneOutEncoder( sigma=0.05) #sigma
lou_enc.fit_transform(df[col_cat],df['Class_enc'])

,drinks
1,0.182522
4,0.185491
6,0.191104
8,0.194611
10,0.158827
...,...
59940,0.195877
59943,0.178579
59944,0.153704
59945,0.197224


In [ ]:
lou_enc.mapping

In [150]:
lou_enc = LeaveOneOutEncoder()
lou_enc.fit_transform(df[col_cat],df['essay_length'])

,drinks
1,3.136048
4,3.136075
6,3.136049
8,3.136060
10,3.145293
...,...
59940,3.136040
59943,3.138056
59944,3.145347
59945,3.136052


In [151]:
lou_enc.mapping

{'drinks':                          sum  count
 desperately       645.232955    210
 drinks_missing   3579.626046   1208
 not_at_all       6784.635061   2157
 often           10710.522351   3413
 rarely          12596.767259   3981
 socially        86307.461805  27521
 very_often        930.146539    319}

## M-estimate Encoder

In [152]:
from category_encoders import MEstimateEncoder

In [153]:
mest_enc = MEstimateEncoder(m=10) # m , #sigma # randomized
mest_enc.fit_transform(df[col_cat],df['Class_enc'])

,drinks
1,0.189199
4,0.189199
6,0.189199
8,0.189199
10,0.148060
...,...
59940,0.189199
59943,0.195398
59944,0.148060
59945,0.189199


In [154]:
mest_enc.mapping

{'drinks': drinks
  1    0.189199
  2    0.148060
  3    0.170094
  4    0.195398
  5    0.179677
  6    0.139352
  7    0.172031
 -1    0.184674
 -2    0.184674
 dtype: float64}

## Weight of Evidence Encoder

In [155]:
from category_encoders import WOEEncoder

In [156]:
woe_enc = WOEEncoder() # randomized=False, sigma=0.05, regularization=1.0
woe_enc.fit_transform(df[col_cat],df['Class_enc'])

,drinks
1,0.029719
4,0.029719
6,0.029719
8,0.029719
10,-0.263874
...,...
59940,0.029719
59943,0.070806
59944,-0.263874
59945,0.029719


In [157]:
woe_enc.mapping

{'drinks': drinks
  1    0.029719
  2   -0.263874
  3   -0.099262
  4    0.070806
  5   -0.030446
  6   -0.328956
  7   -0.069086
 -1    0.000000
 -2    0.000000
 dtype: float64}